In [34]:
import pandas as pd
from datetime import datetime

In [35]:
def date_parser(date):
    return datetime.strptime(date, '%d/%m/%Y')

In [36]:
df = pd.read_csv('data_v5.csv', parse_dates=['date'], date_parser=date_parser)

/var/folders/0b/z_kjvwcj2213940f5fd4ygg00000gp/T/ipykernel_86683/3405913887.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv('data_v5.csv', parse_dates=['date'], date_parser=date_parser)


In [37]:
df.head(3)

,date,campaign_name,spend,currency,trx_value,trx_count,clicks,impr
0,2024-10-01,inbound-us_ppc_gs_mix_bau-always-on_all-web_cg...,252.92,euro,2596.24,14.50,392,3826
1,2024-10-01,inbound-us_ppc_gs_mix_bau-always-on_all-web_cg...,246.09,euro,2914.20,27.35,403,3086
2,2024-10-01,inbound-us_ppc_gs_mix_bau-always-on_all-web_cg...,235.57,euro,1721.70,11.40,414,4138


In [38]:
df.columns

Index(['date', 'campaign_name', 'spend', 'currency', 'trx_value', 'trx_count',
       'clicks', 'impr'],
      dtype='object')

In [39]:
df1 = df.groupby(['date','currency']).agg({'spend': 'sum', 'trx_count': 'sum', 'trx_value': 'sum',
                                            'clicks': 'sum', 'impr': 'sum'})
df1.head()

,,spend,trx_count,trx_value,clicks,impr
date,currency,,,,,
2024-10-01,euro,1190.85,78.42,10853.29,1805,17359
2024-10-02,euro,1094.39,80.61,12752.28,1654,16931
2024-10-03,euro,1123.08,94.52,13651.58,1736,17680
2024-10-04,euro,1198.42,63.11,8820.40,1820,18643
2024-10-05,euro,1142.18,70.06,10255.38,1831,19459


In [80]:
lift = [i*0.01 for i in range(1, 20, 2)]
lift

[0.01, 0.03, 0.05, 0.07, 0.09, 0.11, 0.13, 0.15, 0.17, 0.19]

In [68]:
df1['trx_count'].sum() / df1['impr'].sum()

0.004385713970377237

In [ ]:
atv = df1['trx_value'].sum() / df1['trx_count'].sum()

In [47]:
import numpy as np
def samples_required(df, col1, col2):
    pulled_mean = (df[col1] + df[col2])/2
    effect_size = abs(df[col1] - df[col2]) / np.sqrt(pulled_mean*(1-pulled_mean))
    sample_size = 2 * (1.96 + 0.84)**2  / effect_size**2
    return np.ceil(sample_size)

In [98]:
daily_spend = df1['spend'].sum() / 30

In [99]:
ps = pd.DataFrame()
ps['i2t_cvr_lift'] = lift
ps['i2t_cvr_base'] = [df1['trx_count'].sum() / df1['impr'].sum()] * len(lift)
ps['i2t_cvr_test'] = ps['i2t_cvr_base'] * (1+ps['i2t_cvr_lift'])
ps['min_impr_each'] = samples_required(ps, 'i2t_cvr_base', 'i2t_cvr_test')
ps['duration_days'] = ps['min_impr_each'] * 2 * 30/ df1['impr'].sum()
ps['total_budget'] = ps['duration_days'] * daily_spend
ps['total_trx_base'] = ps['i2t_cvr_base'] * ps['min_impr_each']
ps['total_trx_test'] = ps['i2t_cvr_test'] * ps['min_impr_each']
ps['total_trx_uplift'] = (ps['i2t_cvr_test'] - ps['i2t_cvr_base']) * ps['min_impr_each']
ps['total_gs_uplift'] = ps['total_trx_uplift'] * 155.1

# ps['daily_trx_value_base'] = [df1['trx_value'].mean()]  * len(lift)
# ps['daily_trx_value_test'] = ps['daily_trx_value_base'] * (1+ps['lift'])

# ps['min_trx_count_each'] = samples_required(ps, 'daily_trx_value_base', 'daily_trx_value_test')
# ps['min_duration_in_days']

ps

,i2t_cvr_lift,i2t_cvr_base,i2t_cvr_test,min_impr_each,duration_days,total_budget,total_trx_base,total_trx_test,total_trx_uplift,total_gs_uplift
0,0.01,0.004386,0.004430,35772836.0,4737.810211,4.551120e+06,156889.426605,158458.320871,1568.894266,243335.500665
1,0.03,0.004386,0.004517,4014133.0,531.638037,5.106892e+05,17604.839177,18132.984352,528.145175,81915.316691
2,0.05,0.004386,0.004605,1459261.0,193.266804,1.856513e+05,6399.901354,6719.896422,319.995068,49631.235001
3,0.07,0.004386,0.004693,751752.0,99.563208,9.563999e+04,3296.969249,3527.757096,230.787847,35795.195133
4,0.09,0.004386,0.004780,459137.0,60.808821,5.841269e+04,2013.643555,2194.871475,181.227920,28108.450387
5,0.11,0.004386,0.004868,310284.0,41.094497,3.947520e+04,1360.816874,1510.506730,149.689856,23216.896680
6,0.13,0.004386,0.004956,224252.0,29.700285,2.852996e+04,983.505129,1111.360796,127.855667,19830.413922
7,0.15,0.004386,0.005044,170012.0,22.516655,2.162940e+04,745.624004,857.467604,111.843601,17346.942442
8,0.17,0.004386,0.005131,133588.0,17.692603,1.699544e+04,585.878758,685.478147,99.599389,15447.865209
9,0.19,0.004386,0.005219,107925.0,14.293755,1.373052e+04,473.328180,563.260535,89.932354,13948.508144


In [100]:
ps.to_csv('power_analysis_result.csv', index=False)

In [53]:
from statsmodels.stats.power import TTestIndPower

In [57]:
df1['trx_value'].std()

2666.5041920057097

In [96]:
std_pm

0    0.066244
1    0.066571
2    0.066897
3    0.067221
4    0.067543
5    0.067864
6    0.068183
7    0.068501
8    0.068818
9    0.069132
dtype: float64

In [97]:
# Initialize power analysis object
analysis = TTestIndPower()

# Set parameters
# effect_size = 0.50  # Example: expected 5% uplift in sales
alpha = 0.05        # Significance level
power = 0.8         # Desired power
std_dev = 0.067543   # Estimate of baseline sales variability (in currency units)
uplift = 0.09     # Expected uplift in total sales (as a proportion of baseline)

# Calculate effect size based on expected uplift and standard deviation
# Cohen's d: effect size = mean difference / standard deviation
effect_size = uplift / std_dev

# Compute sample size
sample_size = analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha, alternative='two-sided')
print(f"Required sample size per group: {int(sample_size)}")


Required sample size per group: 9


In [59]:
df1['trx_value'].sum()

308156.57

In [60]:
44645716177 / 308156

144880.2430489752

In [61]:
144880 / 30

4829.333333333333

In [67]:
2790357422 / 308156

9055.01571282078